# REGRESIÓN LINEAL MÚLTIPLE


Crea el fichero u02_p03_a1_reglin_multi_<tus_iniciales>.py. Utiliza pandas y carga los datos del
fichero "50_startups.csv". Utilizaremos como predictoras las columnas "R&D Spend", "Marqueting
Spend" y "State" y como target usaremos "Profit" (beneficios). Imprime los 5 primeros ejemplos de las
predictoras: <br><br>
Para realizar regresión lineal no podemos usar una columna categórica como "State". Codifica sus
valores usando el método one-hot-encoder de manera que aparecerán 3 nuevas columnas que
corresponden a cada uno de los valores que puede tener la columna original (‘New York’, ‘California’,
‘Florida’) y sus valores estarán a 1 cuando sea ese valor y a 0 cuando no. En vez de hacerlo a mano,
vamos a utilizar los objetos sklearn.compose.ColumnTransformer y
sklearn.preprocessing.OneHotEncoder. Busca información de como hacerlo. Tras realizarlo imprime
las primeras 5 filas de los datos transformados.<br><br>
Como todo es un proceso aleatorio, para poder obtener resultados comparables utilizamos la semilla
"123" en todos los procesos. Divide los datos en train y test dejando el 80% para entrenamiento.
Imprime los 5 primeros ejemplos de X_train, y_train, X_test e y_test:

## 1. Imporación de librerías

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [118]:
df = pd.read_csv("recursos/U02_P03/50_Startups.csv")
# Definir las variables predictoras (X) y la variable objetivo (y)
X = df[['R&D Spend', 'Marketing Spend', 'State']]
y = df['Profit']

In [119]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [120]:
print("Primeras 5 filas de las variables predictoras:")
print(X.head())

Primeras 5 filas de las variables predictoras:
   R&D Spend  Marketing Spend       State
0  165349.20        471784.10    New York
1  162597.70        443898.53  California
2  153441.51        407934.54     Florida
3  144372.41        383199.62    New York
4  142107.34        366168.42     Florida


In [121]:
np.random.seed(123)

In [122]:
# Definir las columnas numéricas y categóricas
columnas_numericas = ["R&D Spend", "Marketing Spend"]
columnas_categoricas = ["State"]

# Crear un ColumnTransformer para aplicar OneHotEncoder a la columna "State"
preprocessor = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(), columnas_categoricas)  # Aplicar OneHotEncoder a "State"
    ],
    remainder="passthrough"  # Mantener las demás columnas sin cambios
)

# Aplicar la transformación
X_transformed = preprocessor.fit_transform(df)

# Convertir a DataFrame para ver mejor los resultados
nombres_columnas = preprocessor.get_feature_names_out()

# Renombramos las columnas para eliminar los prefijos 'remainder__' y '__'
nombres_columnas = [col.replace("remainder__", "") for col in nombres_columnas]
df_transformado = pd.DataFrame(X_transformed, columns=nombres_columnas)

# Imprimir las primeras 5 filas
print("Primeras 5 filas de los datos transformados:")
print(df_transformado.head())

Primeras 5 filas de los datos transformados:
   onehot__State_California  onehot__State_Florida  onehot__State_New York  \
0                       0.0                    0.0                     1.0   
1                       1.0                    0.0                     0.0   
2                       0.0                    1.0                     0.0   
3                       0.0                    0.0                     1.0   
4                       0.0                    1.0                     0.0   

   R&D Spend  Administration  Marketing Spend     Profit  
0  165349.20       136897.80        471784.10  192261.83  
1  162597.70       151377.59        443898.53  191792.06  
2  153441.51       101145.55        407934.54  191050.39  
3  144372.41       118671.85        383199.62  182901.99  
4  142107.34        91391.77        366168.42  166187.94  


In [123]:
# Definir las variables predictoras (X) y la variable objetivo (y)
X = df_transformado.drop('Profit', axis=1)  # La columna 'Profit' es la variable objetivo y la eliminamos de X
y = df['Profit']  # 'Profit' es la variable objetivo original, sin transformación

# Dividir los datos en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Verificar las primeras filas de X_train para confirmar que los datos están bien
print("\nPrimeros 5 ejemplos de X_train:")
print(X_train.head())


Primeros 5 ejemplos de X_train:
    onehot__State_California  onehot__State_Florida  onehot__State_New York  \
21                       0.0                    0.0                     1.0   
47                       1.0                    0.0                     0.0   
11                       1.0                    0.0                     0.0   
41                       0.0                    1.0                     0.0   
5                        0.0                    0.0                     1.0   

    R&D Spend  Administration  Marketing Spend  
21   78389.47       153773.43        299737.29  
47       0.00       135426.92             0.00  
11  100671.96        91790.61        249744.55  
41   27892.92        84710.77        164470.71  
5   131876.90        99814.71        362861.36  


In [124]:
# Crear un pipeline con el preprocesador y el modelo de regresión lineal
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Entrenar el modelo con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Hacer predicciones y obtener el score de entrenamiento y prueba
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

ValueError: A given column is not a column of the dataframe

In [ ]:
print(f"\nScore en datos de entrenamiento: {train_score}")
print(f"Score en datos de prueba: {test_score}")

In [ ]:
# Funciones para calcular SSE, SST y SSR
def calculate_SSE(y_true, y_pred):
    """ Calcula el SSE (Sum of Squared Errors) """
    return np.sum((y_true - y_pred) ** 2)

def calculate_SST(y_true):
    """ Calcula el SST (Total Sum of Squares) """
    return np.sum((y_true - np.mean(y_true)) ** 2)

def calculate_SSR(SST, SSE):
    """ Calcula el SSR (Sum of Squares for Regression) """
    return SST - SSE

# Calcular las métricas para los datos de entrenamiento
y_train_pred = pipeline.predict(X_train)
SSE_train = calculate_SSE(y_train, y_train_pred)
SST_train = calculate_SST(y_train)
SSR_train = calculate_SSR(SST_train, SSE_train)

# Calcular R² para entrenamiento
R2_train = 1 - SSE_train / SST_train

# Calcular R² ajustado para entrenamiento
n_train = len(y_train)
k_train = X_train.shape[1]
R2_adj_train = 1 - (SSE_train / (n_train - k_train - 1)) / (SST_train / (n_train - 1))

In [ ]:
# Imprimir las métricas para los datos de entrenamiento
print(f"Entrenamiento:")
print(f"SSE: {SSE_train}")
print(f"SST: {SST_train}")
print(f"SSR: {SSR_train}")
print(f"R²: {R2_train}")
print(f"R² ajustado: {R2_adj_train}")

In [ ]:
# Calcular las métricas para los datos de prueba
y_test_pred = pipeline.predict(X_test)
SSE_test = calculate_SSE(y_test, y_test_pred)
SST_test = calculate_SST(y_test)
SSR_test = calculate_SSR(SST_test, SSE_test)

# Calcular R² para prueba
R2_test = 1 - SSE_test / SST_test

# Calcular R² ajustado para prueba
n_test = len(y_test)
k_test = X_test.shape[1]
R2_adj_test = 1 - (SSE_test / (n_test - k_test - 1)) / (SST_test / (n_test - 1))

In [ ]:
# Imprimir las métricas para los datos de prueba
print(f"\nPrueba:")
print(f"SSE: {SSE_test}")
print(f"SST: {SST_test}")
print(f"SSR: {SSR_test}")
print(f"R²: {R2_test}")
print(f"R² ajustado: {R2_adj_test}")

In [ ]:
# Incluir "Administration" como una nueva característica
columnas_nuevas = ["R&D Spend", "Administration", "Marketing Spend", "State"]

# Crear el nuevo DataFrame con las características
X_nuevas = df[columnas_nuevas]
y_nuevos = df['Profit']

# Dividir nuevamente los datos en entrenamiento y prueba
X_train_nuevas, X_test_nuevas, y_train_nuevos, y_test_nuevos = train_test_split(X_nuevas, y_nuevos, test_size=0.2, random_state=123)

# Crear un nuevo pipeline para el modelo con las nuevas características
preprocessor_nuevas = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(), ["State"]),
    ],
    remainder="passthrough"
)

# Crear el pipeline
pipeline_nuevas = Pipeline(steps=[
    ('preprocessor', preprocessor_nuevas),
    ('regressor', LinearRegression())
])

# Entrenar el modelo con las nuevas características
pipeline_nuevas.fit(X_train_nuevas, y_train_nuevos)

In [ ]:
# Calcular y mostrar las métricas para el nuevo modelo
y_train_pred_nuevas = pipeline_nuevas.predict(X_train_nuevas)
SSE_train_nuevas = calculate_SSE(y_train_nuevos, y_train_pred_nuevas)
SST_train_nuevas = calculate_SST(y_train_nuevos)
SSR_train_nuevas = calculate_SSR(SST_train_nuevas, SSE_train_nuevas)
R2_train_nuevas = 1 - SSE_train_nuevas / SST_train_nuevas
R2_adj_train_nuevas = 1 - (SSE_train_nuevas / (len(y_train_nuevos) - X_train_nuevas.shape[1] - 1)) / (SST_train_nuevas / (len(y_train_nuevos) - 1))

In [ ]:
print(f"\nNuevo modelo de entrenamiento:")
print(f"SSE: {SSE_train_nuevas}")
print(f"SST: {SST_train_nuevas}")
print(f"SSR: {SSR_train_nuevas}")
print(f"R²: {R2_train_nuevas}")
print(f"R² ajustado: {R2_adj_train_nuevas}")

In [ ]:
# Hacer lo mismo para los datos de prueba
y_test_pred_nuevas = pipeline_nuevas.predict(X_test_nuevas)
SSE_test_nuevas = calculate_SSE(y_test_nuevos, y_test_pred_nuevas)
SST_test_nuevas = calculate_SST(y_test_nuevos)
SSR_test_nuevas = calculate_SSR(SST_test_nuevas, SSE_test_nuevas)
R2_test_nuevas = 1 - SSE_test_nuevas / SST_test_nuevas
R2_adj_test_nuevas = 1 - (SSE_test_nuevas / (len(y_test_nuevos) - X_test_nuevas.shape[1] - 1)) / (SST_test_nuevas / (len(y_test_nuevos) - 1))

In [ ]:
print(f"\nNuevo modelo de prueba:")
print(f"SSE: {SSE_test_nuevas}")
print(f"SST: {SST_test_nuevas}")
print(f"SSR: {SSR_test_nuevas}")
print(f"R²: {R2_test_nuevas}")
print(f"R² ajustado: {R2_adj_test_nuevas}")